# Kinetic Monte-Carlo Simulation Notebook
* This notebook is for running a KMC simulation
* Set relevant parameters and create simulation object
* Extract relevent results

### Import modules

In [ ]:
%matplotlib inline
import numpy as np
import time
import matplotlib.pyplot as plt

import ipywidgets as widgets

import KMC_Simulation_v3p3 as KMC_Sim
import KMC_Miscellaneous_v3p3 as KMC_Misc

### Set Parameters

In [ ]:
Parameters = {}

# Simulation Parameters
Parameters['Project_Name'] = 'Detach'      # Name for the project save folder.
Parameters['Simulation_Name'] = 'Ediff0p9'         # Name of particular simulation within a project. Can be an empty string: '' because it's DateTime stamped

Parameters['Number_of_Simulations'] = 10             # number of simulations to perform (fixed parameters)

Parameters['Enable_Print_Outs'] = True             # Enable/Disable simulation prinouts.
Parameters['Enable_Plots'] = False                  # Display final plots at end of simulation. Turn off if running multiple simulations

Parameters['Simulation_Type'] = 'Deposition'       # Simulation Type. Choices: 'Deposition', 'Diffusion (future)'
Parameters['Dimension'] = '3D'                     # Either 2D (1+1) or 3D (2+1). For 2D only Lx is used.

Parameters['Solver_Type'] = 'Binary'            # Type of Solver: 'Linear' O(N^2), 'Binary' O(N log N), 'Set' O(log N)

Parameters['Post_Anneal'] = False                # Run a Post Anneal Step
Parameters['Post_Anneal_Time'] = 1000.0          # Length of Post Anneal Step in seconds.


# Substrate Parameters
Parameters['Lx'] = 1000           # substrate size in x direction
Parameters['Ly'] = 1000           # substrate size in y direction (not used in 2D mode)
Parameters['depth'] = 10          # how many incomplete layers in the lattice (be careful setting this, or errors could occur)

Parameters['Substrate_Type'] = 'Flat'        # Substrate Style . Options: 'Flat', 'Islands', 'Steps'

Parameters['Feature_Layout'] = 'Uniform'        # Feature Layout: 'Uniform': uniform grid, 'Correlated': offset from uniform , 'Random': completely random layout 
Parameters['Feature_Spacing'] = (10,10,0)       # (number in x direction, number in y, avg spread in lattice units)
Parameters['Size_Distribution'] = 'Gaussian'        # Size ditribution of features: 'None', 'Gaussian', or 'Correlated'
Parameters['Size_Values'] = (10,0)               # (Radius or Step Length, Width of distribution)

Parameters['Substrate_Particle_State'] = 'Passive'      # whether substrate particles are active or not. Options: 'Active', 'Passive'.



# Deposition Parameters (Deposition Type simulation only)
Parameters['Pulses'] = 22                      # number of pulses to simulate
Parameters['Dwell_Time'] = 10.0                # Pulse Period
Parameters['Pulse_Shape'] = 'Uniform'           # shape of the deposition pulse. Options: 'Uniform', 'Maxwell'
Parameters['Pulse_Width'] = 1e-5              # Width of deposition pulse in seconds (must be less than dwell time)
Parameters['n'] = int(Parameters['Lx']*Parameters['Ly']/20)       # number of particles deposited each pulse



# Thermal Processes and Activation Energies (Slow Dynamics)
Parameters['Enable_Processes'] = 'No_Uphill_4NN'        # Choose which processes to enable. Options: 'All', 'No_Uphill_4NN', 'No_Detach', 'No_Edge', 'No_Detach_or_Edge'

Parameters['w0'] = 10**6  #10**13             # Prefactor for Arhenius law (Crystal Vibration Frequency: w0 ~ kT/h )
Parameters['Substrate_Temperature'] = 600     # Substrate temperature in Celcius

Parameters['Ea_diffusion'] = 0.9 #1.6114184494753379               # Energy Barrier for surface diffusion in eV
Parameters['Ea_ehrlich_schwoebel'] = 0.0        # Energy Barrier for downhill/uphill diffusion in eV
Parameters['Ea_detach'] = 0.2                  # Energy Barrier for detachment from one nearest neighbor in eV
Parameters['Ea_edge'] = 0.0                    # Energy Barrier for Edge diffusion in eV (3D only). Value of 0 means same as surface diffusion
Parameters['Ea_corner'] = 0.0                   # Energy Barrier for Corner diffusion in eV (3D only)



# Non-Thermal Processes (Fast Dynamics) - currently not enabled
Parameters['Downward_Funneling'] = False      # enables downward funneling when particles land on step edges
Parameters['Transient_Mobility'] = False      # enables transient mobility
Parameters['Island_Chipping'] = False         # enable island chipping



# On-the-Fly Analysis Parameters
Parameters['Coverage'] = True                # do a coverage analysis
Parameters['Island_Analysis'] = True         # do island size distribution analysis
Parameters['Structure_Factor'] = True        # do structure factor analysis
Parameters['Analysis_Time'] = 0.1           # time interval for On-the-Fly analysis
Parameters['On_the_Fly_Save'] = False         # whether to save the on-the-fly results for each simulation. Usually true, unless doing multiple simulations
Parameters['Average_Results'] = True       # average on-the-fly results and save across mulit-simulations. Only used when running multiple simulations



# Save Data Parameters
Parameters['Save_Output_File'] = False         # saves all events in the simulation to output file
Parameters['Save_Lattice'] = False            # save the lattice at specified times. Lattice sites are saved as occupied or unoccupied
Parameters['Save_Surface'] = False            # save a top down view of surface at specified times.
Parameters['Save_Final_Lattice'] = False      # Save a copy of the lattice at end of simulation
Parameters['Save_Times'] = 0.1               # time interval to save lattice and statistics data (set to resolution needed for post analysis). Progress Bar

## Evaluate the Input Parameters
* print elementary process rates

In [ ]:
Rate_Catalog, rates, moves, keys, num_proc = KMC_Misc.Process_Catalog(Parameters).Return_Rates()
Rate_Catalog

In [ ]:
Rate_Catalog['Diff']/0.005

# Run the Simulation
* Pick a simulation script from below

### Single Simulation

In [ ]:
Simulation = KMC_Sim.Simulation(Parameters)   # Initialize the Simulation
Simulation.Run()   # Run the Simulation

In [ ]:
# Simulation.Pulse.Engine.rate_catalog
Simulation.Pulse.Analysis.num_clusters[1][-1]

### Multiple Sims - vary a parameter

In [ ]:
# Vary the edge diffusion energy barrier and evaluate island morphology
E_edge = np.arange(0.1,1.0,0.01)
N = E_edge.shape[0]  # number of sims
print (N)

# Set surface diffusion
Parameters['Ea_diffusion'] = 0.8

Imgs = []  # store surface after each sim
Clusters = []  # store final number of clusters

Parameters['Enable_Plots'] = False
Parameters['Enable_Print_Outs'] = False

for i in range(N):
    # reset parameters
    Parameters['Ea_detach'] = E_edge[i]
    Parameters['Simulation_Name'] = 'rPLD_Ea_'+str(E_edge[i])[:4]+'eV'

    Simulation = KMC_Sim.Simulation(Parameters)
    Simulation.Run()

    num_clusters = Simulation.Pulse.Analysis.num_clusters[1][-1]
    Clusters.append(num_clusters)

    Rate_Catalog = Simulation.Pulse.Engine.rate_catalog
    k1 = Rate_Catalog['Diff']

    # get the final surface
    Lattice = Simulation.Pulse.Engine.lattice
    Surface = np.argmax(Lattice,axis=2)
    Surface = Surface.astype(np.uint8)  # max layer is 256 for this dtype
    Imgs.append(Surface)

    time.sleep(2)  # precaution to prevent same Date-Time Stamp if fast simulation

    print ('Sim',str(i+1),'out of',str(N),'Complete!')

print ('Done!!!!!')

In [ ]:
plt.figure()
plt.plot(Clusters)
# plt.plot(np.gradient(Clusters))

In [ ]:
SaveName = 'Detach_Ediff0p8_Cov0p2'
np.save(SaveName,Clusters)
SaveName2 = SaveName+'Imgs'
np.save(SaveName2,Imgs)

In [ ]:
E_edge[10]

In [ ]:
plt.figure()
plt.plot(Clusters)
# plt.plot(np.gradient(Clusters))

In [ ]:
# view the data
fig, ax = plt.subplots(figsize=(7,7))
im = ax.imshow(Imgs[0],aspect='auto',vmin=0,vmax=2)
plt.tight_layout()
 
def update(i):
#     im.set_data(Imgs[i])
    ax.imshow(Imgs[i],aspect='auto',vmin=0,vmax=2)
    fig.canvas.draw()
     
int_slider = widgets.IntSlider(value=0, min=0, max=N-1, step=1,continuous_update=False)

widgets.interact(update,i=int_slider)

In [ ]:
# Save the data
SaveName = 'Ediff1p1_Eedge0to1'
np.save(SaveName,Imgs)

In [ ]:
SF = Simulation.Pulse.Analysis.structure_factor

In [ ]:
plt.figure(figsize=(10,5))
plt.imshow(SF,aspect='auto',vmin=0,vmax=5)

In [ ]:
plt.figure()
plt.plot(SF[2:,700],'bo')